In [1]:
import pandas as pd

## read_csv

主要涉及相对少使用的engine='python'

**参数支持**:
pandas.read_csv(
    filepath_or_buffer, 
    sep=_NoDefault.no_default, 
    delimiter=None,
    header='infer', 
    names=_NoDefault.no_default, 
    index_col=None, 
    usecols=None, squeeze=None, 
    prefix=_NoDefault.no_default, 
    mangle_dupe_cols=True, 
    dtype=None, 
    **engine=None**, 
    converters=None, 
    true_values=None, 
    false_values=None, 
    skipinitialspace=False, 
    skiprows=None, 
    skipfooter=0, 
    nrows=None, 
    na_values=None, 
    keep_default_na=True, 
    na_filter=True, 
    verbose=False,
    skip_blank_lines=True,
    parse_dates=None, 
    infer_datetime_format=False, 
    keep_date_col=False, 
    date_parser=None, 
    dayfirst=False,
    cache_dates=True, 
    iterator=False, 
    chunksize=None, 
    compression='infer', 
    thousands=None, decimal='.', 
    lineterminator=None, quotechar='"', 
    quoting=0, doublequote=True, 
    escapechar=None, 
    comment=None,
    encoding=None,
    encoding_errors='strict', 
    dialect=None, 
    error_bad_lines=None, 
    warn_bad_lines=None, 
    on_bad_lines=None, 
    delim_whitespace=False, 
    low_memory=True, 
    memory_map=False, 
    float_precision=None, 
    storage_options=None
)

## 问题源

读取的某个amazon欧洲站点的数据, 发现欧洲的一些数字上的标识方式和中国, 美国的有一些差异

1. 小数点, 使用的是","(**逗号**)标识
2. 千分位, 有使用的"."(**点符号**)标识, 也有使用空格符号来标识, 但是需要注意使用的空格符号, 也不是美式键盘输入的空格(unicode = 32), 而是使用的是unicode160和unicode8239(好像还有unicode53).
3. 负号(-), 瑞典的符号使用的是unicode8722, 而不是常用的unicode45

## 处理方式

![图示](https://p0.meituan.net/csc/80c282ac83e8f26a726e22bdfa3e597f30767.png)

1. 小数点中的decimal, 这个没有什么问题, 指定","即可(但是需要注意的是read_excel()不支持这个参数, 但是pandas的doc当中依然提及这个参数)
2. 问题出现在千分位中的空格符号上, 直接通过thousands=chr(8239), 并无法准确处理表格的数据, 带有空格的千分位依然被当成字符串(object)来处理
3. 取出部分的数据, 进行测试, 通过调节**engine**="python", 成功处理这个空格, 但是在使用更多数据进行测试, 发现读取数据错误.
4. 通过调整encoding参数(默认utf-8), 改为西欧字符cp850, 以及其他的几种字符, 发现虽然可以正确读取数据, 但是无法有效处理空格千位符的数字
5. 后面查询发现, 测试的csv文件是带有**BOM**头的文件, 查阅文档发现, pandas处理带有BOM文件的encoding类型中有一个'**utf-8-sig**'的编码方式

![bom](https://p0.meituan.net/csc/1d2bc839498f103c3736f44faffd02db115914.png)

In [11]:
filepath = '法国Test.csv'
df = pd.read_csv(filepath, header=7, thousands=chr(8239), decimal=',', encoding='utf-8-sig', engine="python")

In [6]:
df.info()

# 注意total项, 是数字项

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   date/heure                             404 non-null    object 
 1   numéro de versement                    404 non-null    int64  
 2   type                                   396 non-null    object 
 3   numéro de la commande                  399 non-null    object 
 4   sku                                    383 non-null    object 
 5   description                            404 non-null    object 
 6   quantité                               391 non-null    float64
 7   Marketplace                            390 non-null    object 
 8   traitement                             372 non-null    object 
 9   ville d'où provient la commande        372 non-null    object 
 10  Région d'où provient la commande       61 non-null     object 
 11  code p

In [12]:
# 数字
df.loc[218, 'total']

-6746.45

直接用utf-8处理, 将会出现报错

In [7]:
df = pd.read_csv(filepath, header=7, thousands=chr(8239), decimal=',', encoding='utf-8', engine="python")

ValueError: substring not found

In [8]:
df = pd.read_csv(filepath, header=7, thousands=chr(8239), decimal=',', encoding='utf-8')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   date/heure                             404 non-null    object 
 1   numéro de versement                    404 non-null    int64  
 2   type                                   396 non-null    object 
 3   numéro de la commande                  399 non-null    object 
 4   sku                                    383 non-null    object 
 5   description                            404 non-null    object 
 6   quantité                               391 non-null    float64
 7   Marketplace                            390 non-null    object 
 8   traitement                             372 non-null    object 
 9   ville d'où provient la commande        372 non-null    object 
 10  Région d'où provient la commande       61 non-null     object 
 11  code p

可以看到带有空格的数字没有被处理为float, 在不指定engine='python'时

In [10]:
# 文本

df.loc[218, 'total']

'-6\u202f746,45'

In [ ]:
## 关于engine

engine{‘c’, ‘python’, ‘pyarrow’}, optional

默认c, 读取速度最快

python功能最全